# Prepare Dataset for Fine-Tuning of the heig-vd model

In [ ]:
import os

projsoilsroot = "/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/proj-soils"
config_train_gt_10cm = os.path.join(projsoilsroot, "config", "train", "config-train_gt-10cm.yaml")
config_train_gt_20cm = os.path.join(projsoilsroot, "config", "train", "config-train_gt-20cm.yaml")
config_train_gt_40cm = os.path.join(projsoilsroot, "config", "train", "config-train_gt-40cm.yaml")
config_train_scratch_10cm = os.path.join(projsoilsroot, "config", "train", "config-train_scratch-10cm.yaml")
config_train_scratch_20cm = os.path.join(projsoilsroot, "config", "train", "config-train_scratch-20cm.yaml")
config_train_scratch_40cm = os.path.join(projsoilsroot, "config", "train", "config-train_scratch-40cm.yaml")

## GT

### 1. Snap to 10cm

In [ ]:
import os
import numpy as np
import geopandas as gpd


root = "/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/GT/20240216/0-0-0-raw"
out_folder = "/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/GT/20240216/0-0-snapped"

for file in os.listdir(root):
    if not file.endswith(".shp"):
        continue

    gt = gpd.read_file(os.path.join(root, file))
    bbox = list(gt.total_bounds)

    deviation_from_snap = np.array(bbox).round(1) - bbox
    plus_xmin, plus_ymin, _, _ = deviation_from_snap

    gt.geometry = gt.geometry.translate(plus_xmin, plus_ymin)
    gt.to_file(os.path.join(out_folder, file.replace(".shp", ".gpkg")))


### 2. Assign classes

In [ ]:
source_folder = "/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/GT/20240216/0-0-snapped"
out_folder = "/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/GT/20240216/0-cleaned"

In [ ]:
class_mapping = {
    np.nan: 0,
    "batiment": 1,
    "toit_vegetalise": 2,
    "surface_non_beton": 3,
    "surface_beton": 4,
    "eau_bassin": 5,
    "roche_dure_meuble": 6,
    "eau_naturelle": 7,
    "roseliere": 8,
    "sol_neige": 9,
    "sol_vegetalise": 10,
    "surface_riparienne": 11,
    "sol_divers": 12,
    "sol_vigne": 13,
    "sol_agricole": 14,
    "sol_bache": 15,
    "sol_serre_temporaire": 16,
    "serre_permanente": 17
}

soil_classes = [9, 10, 12, 13, 14, 15, 16]

package_mapping = {
    0: 0, 1: 1, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 6, 9: 7, 10: 8,
    11: 8, 12: 8, 13: 9, 14: 10, 15: 10, 16: 10, 17: 11
}

cl12_mapping = {
    0: 0, 
    1: 1, # batiment
    2: 1, # toit_vegetalise -> batiment
    3: 2, # surface_non_beton
    4: 3, # surface_beton
    5: 5, # eau_bassin -> eau_naturelle
    6: 4, # roche_dure_meuble
    7: 5, # eau_naturelle
    8: 6, # roseliere
    9: 7, # sol_neige
    10: 8, # sol_vegetalise
    11: 8, # surface_riparienne -> sol_vegetalise
    12: 9, # sol_divers
    13: 10, # sol_vigne
    14: 11, # sol_agricole
    15: 12, # sol_bache
    16: 12, # sol_serre_temporaire -> sol_bache
    17: 1, # serre_permanente -> batiment
    }


for root, dir, files in os.walk(source_folder):
        for file in files:
            if not file.endswith((".shp", ".gpkg")):
                continue
            
            gt = gpd.read_file(os.path.join(root, file))

            gt["CLASSE_SEN_ID"] = gt.apply(
                lambda x: class_mapping[x["CLASSE_SEN"]], axis=1)
            gt["package_id"] = gt.apply(
                lambda x: package_mapping[x["CLASSE_SEN_ID"]], axis=1)
            gt["cl12_id"] = gt.apply(
                lambda x: cl12_mapping[x["CLASSE_SEN_ID"]], axis=1)
            gt["SOIL"] = gt.apply(
                lambda x: x["CLASSE_SEN_ID"] in soil_classes, axis=1)


            # gt["geometry"] = gt.make_valid()

            gt = gt.dropna(subset = ["CLASSE_SEN_ID"])
            gt = gt.explode(index_parts=False)
            gt = gt.loc[gt["geometry"].geom_type=='Polygon']

            gt = gt[['CLASSE_SEN', 'CLASSE_SEN_ID', 'package_id', 'cl12_id', 'SOIL', 'geometry']]

            if file.endswith(".shp"):
                gt.to_file(os.path.join(out_folder, file.rstrip(".shp")+".gpkg"), driver="GPKG")
            else:
                gt.to_file(os.path.join(out_folder, file), driver="GPKG")

In [ ]:
gt.head()

### 3. Rasterize

In [ ]:
! python utilities/rasterize_gt.py --config_file {config_train_gt_10cm}

### 4. Rescale

10cm -> 20cm

In [ ]:
! python utilities/rescale_tif.py --config_file {config_train_gt_20cm}

10cm -> 40cm

In [ ]:
! python utilities/rescale_tif.py --config_file {config_train_gt_40cm}

### 4. Cut tiff to Grid

10cm

In [ ]:
! python utilities/cut_tiff_to_grid.py --config_file {config_train_gt_10cm}

20cm

In [22]:
! python utilities/cut_tiff_to_grid.py --config_file {config_train_gt_20cm}

2024-03-08 09:47:14.330 | INFO     | __main__:<module>:258 - TIFF_FOLDER = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/GT/20240216/3-rasterized-12cl/20cm'
2024-03-08 09:47:14.331 | INFO     | __main__:<module>:259 - OUT_FOLDER = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/GT/20240216/4-cut-to-grid-12cl/20cm/512px'
2024-03-08 09:47:14.331 | INFO     | __main__:<module>:260 - GRID_PATH = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/heig-vd_finetuned/recursive_grids_max204-8m.gpkg'
2024-03-08 09:47:14.331 | INFO     | __main__:<module>:261 - GRID_QUERY = 'depth == 1'
2024-03-08 09:47:14.331 | INFO     | __main__:<module>:262 - CELL_LENGTH = 512
2024-03-08 09:47:14.331 | INFO     | __main__:<module>:263 - ID_COLUMN = 'str_ids'
2024-03-08 09:47:14.331 | INFO     | __main__:<module>:264 - LOG_FILE = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/proj-soils/logs/cut_tiff_to_grid.log'
2024-03-08 09:47:14.331 | INFO     | __main__:<modu

40cm

In [23]:
! python utilities/cut_tiff_to_grid.py --config_file {config_train_gt_40cm}

2024-03-08 09:47:19.746 | INFO     | __main__:<module>:258 - TIFF_FOLDER = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/GT/20240216/3-rasterized-12cl/40cm'
2024-03-08 09:47:19.746 | INFO     | __main__:<module>:259 - OUT_FOLDER = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/GT/20240216/4-cut-to-grid-12cl/40cm/512px'
2024-03-08 09:47:19.746 | INFO     | __main__:<module>:260 - GRID_PATH = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/heig-vd_finetuned/recursive_grids_max204-8m.gpkg'
2024-03-08 09:47:19.746 | INFO     | __main__:<module>:261 - GRID_QUERY = 'depth == 2'
2024-03-08 09:47:19.746 | INFO     | __main__:<module>:262 - CELL_LENGTH = 512
2024-03-08 09:47:19.746 | INFO     | __main__:<module>:263 - ID_COLUMN = 'str_ids'
2024-03-08 09:47:19.746 | INFO     | __main__:<module>:264 - LOG_FILE = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/proj-soils/logs/cut_tiff_to_grid.log'
2024-03-08 09:47:19.747 | INFO     | __main__:<modu

## Scratch

### 1. RGBI -> RGB

In [ ]:
! python utilities/rgbi2rgb.py --config_file {config_train_scratch_10cm}

### 2. Rescale

10cm -> 10cm

In [ ]:
! python utilities/rescale.py --config_file {config_train_scratch_10cm}

10cm -> 20cm

In [ ]:
! python utilities/rescale.py --config_file {config_train_scratch_20cm}

10cm -> 40cm

In [ ]:
! python utilities/rescale.py --config_file {config_train_scratch_40cm}

### 3. Cut tiff to grid

10cm

In [ ]:
! python utilities/cut_tiff_to_grid.py --config_file {config_train_scratch_10cm}

20cm


In [20]:
! python utilities/cut_tiff_to_grid.py --config_file {config_train_scratch_20cm}

2024-03-08 09:46:13.924 | INFO     | __main__:<module>:258 - TIFF_FOLDER = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/scratch/horizontal_scratch_mosaics-rgb/2-rescaled/20cm'
2024-03-08 09:46:13.925 | INFO     | __main__:<module>:259 - OUT_FOLDER = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/scratch/horizontal_scratch_mosaics-rgb/3-cut_to_grid/20cm/512px'
2024-03-08 09:46:13.925 | INFO     | __main__:<module>:260 - GRID_PATH = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/heig-vd_finetuned/recursive_grids_max204-8m.gpkg'
2024-03-08 09:46:13.925 | INFO     | __main__:<module>:261 - GRID_QUERY = 'depth == 1'
2024-03-08 09:46:13.925 | INFO     | __main__:<module>:262 - CELL_LENGTH = 512
2024-03-08 09:46:13.925 | INFO     | __main__:<module>:263 - ID_COLUMN = 'str_ids'
2024-03-08 09:46:13.925 | INFO     | __main__:<module>:264 - LOG_FILE = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/proj-soils/logs/cut_tiff_to_grid.log'
2024-03-0

40cm


In [21]:
! python utilities/cut_tiff_to_grid.py --config_file {config_train_scratch_40cm}

2024-03-08 09:46:24.566 | INFO     | __main__:<module>:258 - TIFF_FOLDER = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/scratch/horizontal_scratch_mosaics-rgb/2-rescaled/40cm'
2024-03-08 09:46:24.566 | INFO     | __main__:<module>:259 - OUT_FOLDER = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/scratch/horizontal_scratch_mosaics-rgb/3-cut_to_grid/40cm/512px'
2024-03-08 09:46:24.566 | INFO     | __main__:<module>:260 - GRID_PATH = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/heig-vd_finetuned/recursive_grids_max204-8m.gpkg'
2024-03-08 09:46:24.566 | INFO     | __main__:<module>:261 - GRID_QUERY = 'depth == 2'
2024-03-08 09:46:24.566 | INFO     | __main__:<module>:262 - CELL_LENGTH = 512
2024-03-08 09:46:24.566 | INFO     | __main__:<module>:263 - ID_COLUMN = 'str_ids'
2024-03-08 09:46:24.566 | INFO     | __main__:<module>:264 - LOG_FILE = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/proj-soils/logs/cut_tiff_to_grid.log'
2024-03-0

## Create Dataset

10cm

In [24]:
! python utilities/random_split.py --config_file {config_train_gt_10cm}

2024-03-08 09:50:03.401 | INFO     | __main__:<module>:156 - SOURCE_IPT_FOLDER = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/scratch/horizontal_scratch_mosaics-rgb/3-cut_to_grid/10cm/512px'
2024-03-08 09:50:03.401 | INFO     | __main__:<module>:157 - SOURCE_TGT_FOLDER = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/GT/20240216/4-cut-to-grid-12cl/10cm/512px'
2024-03-08 09:50:03.402 | INFO     | __main__:<module>:158 - TARGET_ROOT = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/datasets.nosync/dataset_12cl_seed6-adjusted_multiscale'
2024-03-08 09:50:03.402 | INFO     | __main__:<module>:159 - SEED = None
2024-03-08 09:50:03.402 | INFO     | __main__:<module>:160 - SPLIT_FILE = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/datasets.nosync/seed6_adjusted_split.yaml'
2024-03-08 09:50:03.402 | INFO     | __main__:<module>:161 - LOG_FILE = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/proj-soils/logs/random_split.log'
202

20cm

In [25]:
! python utilities/random_split.py --config_file {config_train_gt_20cm}

2024-03-08 09:50:10.880 | INFO     | __main__:<module>:156 - SOURCE_IPT_FOLDER = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/scratch/horizontal_scratch_mosaics-rgb/3-cut_to_grid/20cm/512px'
2024-03-08 09:50:10.880 | INFO     | __main__:<module>:157 - SOURCE_TGT_FOLDER = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/GT/20240216/4-cut-to-grid-12cl/20cm/512px'
2024-03-08 09:50:10.880 | INFO     | __main__:<module>:158 - TARGET_ROOT = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/datasets.nosync/dataset_12cl_seed6-adjusted_multiscale'
2024-03-08 09:50:10.880 | INFO     | __main__:<module>:159 - SEED = None
2024-03-08 09:50:10.880 | INFO     | __main__:<module>:160 - SPLIT_FILE = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/datasets.nosync/seed6_adjusted_split.yaml'
2024-03-08 09:50:10.880 | INFO     | __main__:<module>:161 - LOG_FILE = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/proj-soils/logs/random_split.log'
202

40cm

In [26]:
! python utilities/random_split.py --config_file {config_train_gt_40cm}

2024-03-08 09:50:13.007 | INFO     | __main__:<module>:156 - SOURCE_IPT_FOLDER = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/scratch/horizontal_scratch_mosaics-rgb/3-cut_to_grid/40cm/512px'
2024-03-08 09:50:13.007 | INFO     | __main__:<module>:157 - SOURCE_TGT_FOLDER = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/GT/20240216/4-cut-to-grid-12cl/40cm/512px'
2024-03-08 09:50:13.007 | INFO     | __main__:<module>:158 - TARGET_ROOT = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/datasets.nosync/dataset_12cl_seed6-adjusted_multiscale'
2024-03-08 09:50:13.007 | INFO     | __main__:<module>:159 - SEED = None
2024-03-08 09:50:13.007 | INFO     | __main__:<module>:160 - SPLIT_FILE = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/data/datasets.nosync/seed6_adjusted_split.yaml'
2024-03-08 09:50:13.007 | INFO     | __main__:<module>:161 - LOG_FILE = '/Users/nicibe/Desktop/Job/swisstopo_stdl/soil_fribourg/proj-soils/logs/random_split.log'
202